#**A One-Class Classification decision Tree based on kernel density estimation**

In [1]:
# importing necessary libraries
import math
import warnings
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import classification_report

In [2]:
warnings.filterwarnings("ignore") #to avoid warnings

In [3]:
letters = pd.read_csv("letter-recognition.csv")

In [4]:
letters.columns = ['label', 'xbox', 'ybox', 'width', 'height', 'onpix', 'xbar','ybar', 'x2bar', 'y2bar', 'xybar', 'x2ybar', 'xy2bar', 'xedge','xedgey', 'yedge', 'yedgex']


In [5]:
letters

,label,xbox,ybox,width,height,onpix,xbar,ybar,x2bar,y2bar,xybar,x2ybar,xy2bar,xedge,xedgey,yedge,yedgex
0,I,5,12,3,7,2,10,5,5,4,13,3,9,2,8,4,10
1,D,4,11,6,8,6,10,6,2,6,10,3,7,3,7,3,9
2,N,7,11,6,6,3,5,9,4,6,4,4,10,6,10,2,8
3,G,2,1,3,1,1,8,6,6,6,6,5,9,1,7,5,10
4,S,4,11,5,8,3,8,8,6,9,5,6,6,0,8,9,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19994,D,2,2,3,3,2,7,7,7,6,6,6,4,2,8,3,7
19995,C,7,10,8,8,4,4,8,6,9,12,9,13,2,9,3,7
19996,T,6,9,6,7,5,6,11,3,7,11,9,5,2,12,2,4
19997,S,2,3,4,2,1,8,7,2,6,10,6,8,1,9,5,8


In [6]:
class Node:
    def __init__(self, L, R, a_prime):
        self.L = L
        self.R = R
        self.a_prime = a_prime
        self.children = []

In [7]:
class Tree:
    def __init__(self, X, y, gamma, alpha, beta, nu):
        self.gamma = gamma
        self.alpha = alpha
        self.beta = beta
        self.nu = nu
        self.n_samples, self.n_features = X.shape
        self.A = X.columns
        eligible = dict((a, True) for a in self.A)
        X = X
        self.root = self.build_tree(float('-inf'), float('inf'), X, 0, eligible)
    
    def build_tree(self, left, right, X, prev_A_tr, eligible):
        
        # 2.5 Stopping Conditions
        n_t = len(X)

        if n_t == 0:
            return None

        min_proxy_impurity_decrease = float('inf')
        a_t = None
        children = None
        Y_t = None

        for a in self.A:
            # 2.4 Pre-pruning mechanism
            if eligible[a] == False:
                continue

            if len(np.unique(X[a])) == 1:
                continue
            
            # 2.1 Density Estimation
            f, x = self.kernelDensityEstimator(X, a)

            # 2.2 Division
            # (a) Clipping KDE
            threshold = self.gamma * max(f)

            Y = []
            i = 0
            while i < len(f):
                while i < len(f) and f[i] < threshold:
                    i += 1

                if i == len(f):
                    break

                l = i
                i += 1

                while i < len(f) and f[i] >= threshold:
                    i += 1
                
                r = i - 1
                Y.append([l, r])
                i += 1

            # (b) Revision
            Y_prime = []
            for [l, r] in Y:
                i = l + 1
                L, R = l, r
                while i < r:
                    while i < r and f[i] < f[i + 1]:
                        i += 1
                    
                    if i == r:
                        continue
                    
                    maxima1 = i
                    i += 1

                    while i < r and f[i] > f[i + 1]:
                        i += 1
                    
                    if i == r:
                        continue

                    minima = i
                    i += 1

                    while i < r and f[i] < f[i + 1]:
                        i += 1

                    if i == r:
                        continue

                    maxima2 = i
                    
                    if f[minima] <= alpha * min(f[maxima1], f[maxima2]):
                        Y_prime.append([L, i])
                        L = i
                Y_prime.append([L, R])
            

            # (c) Assessment
            Y_prime[:] = [[l, r] for [l,r] in Y_prime if len(X.loc[(x[l] <= X[a]) & (X[a] <= x[r])]) > self.beta * self.n_samples]

            Y_prime_prime = []
            for [l, r] in Y_prime:
                sub_interval = X.loc[(x[l] <= X[a]) & (X[a] <= x[r])]
                Y_prime_prime.append([sub_interval[a].min(), sub_interval[a].max()])

            # 2.3 Impurity Decrease
            proxy_impurity_decrease = 0
            for [l_val, r_val] in Y_prime_prime:
                # print(l_val, r_val)
                n_t_i = len(X.loc[(l_val <= X[a]) & (X[a] <= r_val)])
                n_t_i_prime = n_t_i * (r_val - l_val) / (X[a].max() - X[a].min())
                proxy_impurity_decrease += (n_t_i * n_t_i_prime) / (n_t_i + n_t_i_prime)


            if proxy_impurity_decrease < min_proxy_impurity_decrease:
                min_proxy_impurity_decrease = proxy_impurity_decrease
                a_t = a
                Y_t = Y_prime_prime

        if a_t and len(Y_t) > 0:
            A_tr = len(X.loc[(Y_t[0][0] <= X[a]) & (X[a] <= Y_t[-1][1])]) / len(X)
            if A_tr == prev_A_tr:
                return None
            root = Node(left, right, a_t)
            eligible[a] = False
            for [l, r] in Y_t:
                child = self.build_tree(l, r, X.loc[(l <= X[a]) & (X[a] <= r)], A_tr, eligible)
                if child:
                    root.children.append(child)
        else:
            root = None

        return root

    # FUNCTION FOR KERNEL DENSITY ESTIMATION
    def kernelDensityEstimator(self, X, a):
        n_t = len(X)
        Q1 = X[a].quantile(0.25)
        Q3 = X[a].quantile(0.75)
        IQR = Q3 - Q1

        if IQR != 0:
            h_t = 0.9 * min(X[a].std(), IQR / 1.34) * (n_t ** (-1 / 5))
        else:
            h_t = 0.9 * X[a].std() * (n_t ** (-1 / 5))

        lo, hi = X[a].min(), X[a].max()
        x, f = [], []
        density_points = 512
        increment = (hi - lo) / density_points

        i = lo
        while i <= hi:
            x.append(i)
            i += increment

        for z in x:
            f.append(0)
            for x_i in X[a]:
                f[-1] += self.K((z - x_i) / h_t)
            f[-1] = (1 / (n_t * h_t)) * f[-1]

        return f, x
    

    def K(self, y):
        return 1 / math.sqrt(2 * math.pi) * math.exp(-(y ** 2) / 2)
    

    def predict(self, X):
        predictions = []
        for i, x in X.iterrows():
            copy = self.root
            predictions.append(self.make_prediction(x, copy))

        return predictions
    
    def make_prediction(self, x, root):
        if len(root.children) == 0:
            return 1

        for child in root.children:
            if root.L <= x[root.a_prime] <= root.R:
                return self.make_prediction(x, child)

        return 0

In [8]:
class oneClassClassification:
    def __init__(self, alpha, beta, gamma, nu):
        self.gamma = gamma
        self.alpha = alpha
        self.beta = beta
        self.nu = nu

    def fitFunc(self, X, y):
        self.tree = Tree(X, y, gamma, alpha, beta, nu)

    def predict(self, X):
        return self.tree.predict(X)

In [9]:
# TRAINING OF DATASETS FOR DIFFERENT NOISE LEVELS
target_class = 'A'
letters['label'] = (letters['label'] == target_class).astype(int)

print("DATASET -------------")
print()
print(letters)

DATASET -------------

       label  xbox  ybox  width  height  ...  xy2bar  xedge  xedgey  yedge  yedgex
0          0     5    12      3       7  ...       9      2       8      4      10
1          0     4    11      6       8  ...       7      3       7      3       9
2          0     7    11      6       6  ...      10      6      10      2       8
3          0     2     1      3       1  ...       9      1       7      5      10
4          0     4    11      5       8  ...       6      0       8      9       7
...      ...   ...   ...    ...     ...  ...     ...    ...     ...    ...     ...
19994      0     2     2      3       3  ...       4      2       8      3       7
19995      0     7    10      8       8  ...      13      2       9      3       7
19996      0     6     9      6       7  ...       5      2      12      2       4
19997      0     2     3      4       2  ...       8      1       9      5       8
19998      1     4     9      6       6  ...       8      2     

In [10]:
# DIVIDING THE DATA INTO TWO CLASS
# 1. Target Class Data - Label Field 1
# 2. Outlier Data - Label FIeld 0

targetClassData = letters.loc[letters['label'] == 1]
print("---------- Target Class Data ----------")
print()
print(targetClassData)

outlierData = letters.loc[letters['label'] == 0]
print("---------- Outlier Data ----------")
print()
print(outlierData)

---------- Target Class Data ----------

       label  xbox  ybox  width  height  ...  xy2bar  xedge  xedgey  yedge  yedgex
6          1     1     1      3       2  ...       8      1       6      2       7
77         1     3     7      5       5  ...       9      2       6      3       8
117        1     3     8      5       6  ...       8      2       6      3       7
129        1     2     1      4       2  ...       8      2       5      2       7
133        1     3     7      5       5  ...       9      2       4      2       7
...      ...   ...   ...    ...     ...  ...     ...    ...     ...    ...     ...
19892      1     3     7      4       5  ...       8      2       7      1       7
19950      1     6    10      7       6  ...      12      4       4      3      11
19965      1     2     3      3       1  ...       8      1       6      1       7
19976      1     3     9      5       6  ...       8      2       7      2       7
19998      1     4     9      6       6  ...  

In [11]:
# TESTING OF DATASETS FOR DIFFERENT NOISE LEVELS

In [12]:
# Initializing the constants
alpha = 0.5
beta = 0.02
gamma = 0.05
nu = 0.05

In [13]:
# FOR NOISE LEVEL = 2%
# TRAINING
outlierPercent = 2
m = len(targetClassData)
numOfOutliers = m*outlierPercent 
includedOutliers = outlierData.sample(n = numOfOutliers)

data = targetClassData.append(includedOutliers)
print(data)

x = data.drop('label',1)
y = data['label']

trainingX,testingX,trainingY,testingY = tts(x,y,test_size = 0.33,random_state = 729)

       label  xbox  ybox  width  height  ...  xy2bar  xedge  xedgey  yedge  yedgex
6          1     1     1      3       2  ...       8      1       6      2       7
77         1     3     7      5       5  ...       9      2       6      3       8
117        1     3     8      5       6  ...       8      2       6      3       7
129        1     2     1      4       2  ...       8      2       5      2       7
133        1     3     7      5       5  ...       9      2       4      2       7
...      ...   ...   ...    ...     ...  ...     ...    ...     ...    ...     ...
3748       0     2     7      3       5  ...      14      0       8      7       8
1887       0     5     9      5       7  ...      10      3       8      4       6
4557       0     3     7      4       5  ...       8      2       8      4       8
18351      0     4     9      5       7  ...       8      2       7      6      11
969        0     1     0      1       1  ...       7      0       8      2       8

[23

In [14]:
# FOR NOISE LEVEL = 2%
# TESTING

classify = oneClassClassification(alpha,beta,gamma,nu)
classify.fitFunc(trainingX,trainingY)

predictedY = classify.predict(testingX)
report = classification_report(testingY,predictedY)

print("Classification report for noise level = 2 % ----------")
print()
print()
print(report)
print()

Classification report for noise level = 2 % ----------


              precision    recall  f1-score   support

           0       0.00      0.00      0.00       544
           1       0.30      1.00      0.47       238

    accuracy                           0.30       782
   macro avg       0.15      0.50      0.23       782
weighted avg       0.09      0.30      0.14       782




In [15]:
# FOR NOISE LEVEL = 5%
# TRAINING
outlierPercent = 5
m = len(targetClassData)
numOfOutliers = m*outlierPercent 
includedOutliers = outlierData.sample(n = numOfOutliers)

data = targetClassData.append(includedOutliers)
print(data)

x = data.drop('label',1)
y = data['label']

trainingX,testingX,trainingY,testingY = tts(x,y,test_size = 0.33,random_state = 729)

       label  xbox  ybox  width  height  ...  xy2bar  xedge  xedgey  yedge  yedgex
6          1     1     1      3       2  ...       8      1       6      2       7
77         1     3     7      5       5  ...       9      2       6      3       8
117        1     3     8      5       6  ...       8      2       6      3       7
129        1     2     1      4       2  ...       8      2       5      2       7
133        1     3     7      5       5  ...       9      2       4      2       7
...      ...   ...   ...    ...     ...  ...     ...    ...     ...    ...     ...
18941      0     7    13      8       7  ...       9      6       2      1       9
4679       0     5     7      7       5  ...       8      3       6      5      11
19261      0     4     5      6       7  ...       9      2       7      5      10
18663      0     6    10      8       8  ...      11      3       8      3       7
7608       0     4     2      5       4  ...       8      8       6      2       7

[47

In [16]:
# FOR NOISE LEVEL = 5%
# TESTING

classify = oneClassClassification(alpha,beta,gamma,nu)
classify.fitFunc(trainingX,trainingY)

predictedY = classify.predict(testingX)
report = classification_report(testingY,predictedY)

print("Classification report for noise level = 5 % ----------")
print()
print()
print(report)
print()

Classification report for noise level = 2 % ----------


              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1320
           1       0.16      1.00      0.27       243

    accuracy                           0.16      1563
   macro avg       0.08      0.50      0.13      1563
weighted avg       0.02      0.16      0.04      1563




In [17]:
# FOR NOISE LEVEL = 10%
# TRAINING
outlierPercent = 10
m = len(targetClassData)
numOfOutliers = m*outlierPercent 
includedOutliers = outlierData.sample(n = numOfOutliers)

data = targetClassData.append(includedOutliers)
print(data)

x = data.drop('label',1)
y = data['label']

trainingX,testingX,trainingY,testingY = tts(x,y,test_size = 0.33,random_state = 729)

       label  xbox  ybox  width  height  ...  xy2bar  xedge  xedgey  yedge  yedgex
6          1     1     1      3       2  ...       8      1       6      2       7
77         1     3     7      5       5  ...       9      2       6      3       8
117        1     3     8      5       6  ...       8      2       6      3       7
129        1     2     1      4       2  ...       8      2       5      2       7
133        1     3     7      5       5  ...       9      2       4      2       7
...      ...   ...   ...    ...     ...  ...     ...    ...     ...    ...     ...
16605      0     3     2      5       4  ...       8      2       8      3       8
18277      0     1    10      0       7  ...       8      0       8      0       8
14843      0     5     9      7       7  ...       8      7       7      5       3
2836       0     2     4      4       3  ...       7      1       8      5       8
8695       0     6     8      6       6  ...      11      2       9      4       9

[86

In [18]:
# FOR NOISE LEVEL = 10%
# TESTING

classify = oneClassClassification(alpha,beta,gamma,nu)
classify.fitFunc(trainingX,trainingY)

predictedY = classify.predict(testingX)
report = classification_report(testingY,predictedY)

print("Classification report for noise level = 10 % ----------")
print()
print()
print(report)
print()

Classification report for noise level = 10 % ----------


              precision    recall  f1-score   support

           0       0.00      0.00      0.00      2618
           1       0.09      1.00      0.16       247

    accuracy                           0.09      2865
   macro avg       0.04      0.50      0.08      2865
weighted avg       0.01      0.09      0.01      2865




In [19]:
# FOR NOISE LEVEL = 15%
# TRAINING
outlierPercent = 15
m = len(targetClassData)
numOfOutliers = m*outlierPercent 
includedOutliers = outlierData.sample(n = numOfOutliers)

data = targetClassData.append(includedOutliers)
print(data)

x = data.drop('label',1)
y = data['label']

trainingX,testingX,trainingY,testingY = tts(x,y,test_size = 0.33,random_state = 729)

       label  xbox  ybox  width  height  ...  xy2bar  xedge  xedgey  yedge  yedgex
6          1     1     1      3       2  ...       8      1       6      2       7
77         1     3     7      5       5  ...       9      2       6      3       8
117        1     3     8      5       6  ...       8      2       6      3       7
129        1     2     1      4       2  ...       8      2       5      2       7
133        1     3     7      5       5  ...       9      2       4      2       7
...      ...   ...   ...    ...     ...  ...     ...    ...     ...    ...     ...
16616      0     4     4      6       3  ...       8      7       5      2       8
4985       0     7     9      6       7  ...       7      3      10      1       7
19155      0     9     9     12       8  ...       7     12       9      7       3
3275       0     6     8      6       6  ...       8      4      12      1       7
17060      0     2     3      2       2  ...       6      2       7      4       8

[12

In [20]:
# FOR NOISE LEVEL = 15%
# TESTING

classify = oneClassClassification(alpha,beta,gamma,nu)
classify.fitFunc(trainingX,trainingY)

predictedY = classify.predict(testingX)
report = classification_report(testingY,predictedY)

print("Classification report for noise level = 15 % ----------")
print()
print()
print(report)
print()

Classification report for noise level = 15 % ----------


              precision    recall  f1-score   support

           0       0.00      0.00      0.00      3898
           1       0.06      1.00      0.12       268

    accuracy                           0.06      4166
   macro avg       0.03      0.50      0.06      4166
weighted avg       0.00      0.06      0.01      4166


